In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from portal_analytics.api import AnalyticsApi
import portal_analytics.parser as parser
import portal_analytics.create_rasters as cr
from shapely.geometry import box, Point
import json
import pandas as pd

Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [3]:
with open('items.json') as file:
      items = json.load(file)
layers=  items["layers"]
for layer in layers:
    print(layer)

mineral_occurrences
mines
mineral_tenements
scanned_250K_maps
nvcl
all_boreholes


In [4]:
KEY_FILE = 'GeosciencePortal-eeac25f8efb3.json'
ACCOUNT_NAME = 'AUSGIN website'
PROPERTY_NAME = 'AUSGIN'
PROFILE_NAME = 'AUSGIN - exclude internal traffic'


In [6]:
connection=AnalyticsApi(KEY_FILE,ACCOUNT_NAME,PROPERTY_NAME,PROFILE_NAME)

In [7]:
rows, headers = connection.get_results()

In [9]:
raw_results = parser.create_dataframe(rows, headers)
raw_results.to_excel("raw_results.xlsx")

In [ ]:
raw_results["ga:eventCategory"].unique()

In [ ]:
raw_results["ga:hostname"].unique()

In [ ]:
raw_results[raw_results["ga:eventCategory"]=="New:Help Panel"]


In [ ]:
data_frame = parser.process_dataframe(raw_results)
data_frame["ga:totalEvents"]=data_frame["ga:totalEvents"].apply(int)
add_layers=data_frame[data_frame["ga:eventCategory"]=="Add:KnownLayer"].groupby(by="ga:eventAction")["ga:totalEvents"].sum()

In [ ]:
pd.DataFrame(add_layers).sort_values("ga:totalEvents", ascending=False).to_excel("add_layers.xlsx")

In [ ]:
query=raw_results[raw_results["ga:eventCategory"]=="Query"]

In [ ]:
query["ga:eventAction"].unique();

In [ ]:
gdf = parser.create_geodataframe(data_frame)
%matplotlib inline

In [ ]:
map_centre=Point(133.3,-26)

australia_bounds = box(110,-45,155,-10)
buff=map_centre.buffer(1)


In [ ]:
#gdf["distance"] = gdf["shape"].centroid.distance(map_centre)
gdf["area"] = gdf["shape"].area
gdf["weighting"] = australia_bounds.area/ gdf["shape"].area * gdf["ga:totalEvents"]
gdf = gdf[gdf["shape"].centroid.intersects(australia_bounds)]
#gdf = gdf[gdf["shape"].centroid.intersects(buff)]

gdf = gdf[gdf["area"] <= australia_bounds.area]
#gdf = gdf[gdf["area"] > 0]
#gdf


x=gdf[gdf['ga:eventAction'] == "Layer:Scanned 250K Geological Map Index"]


In [ ]:
cr.create_spatial_data(gdf)



In [ ]:
x=1